In [1]:
import random
import typing
import itertools
from PIL import Image
import weave
weave.use_frontend_devmode()

In [2]:
# Not really model predictions but it'll work for now

ims = []
base_im = Image.linear_gradient('L')
for rotate in range(5):
    for shear in range(5):
        for i in range(random.randrange(10)):
            ims.append({
                'rotate': rotate,
                'shear': shear,
                'y': random.choice(['a', 'b', 'c']),
                'x': random.randrange(10),
                'image': base_im
                    .rotate(rotate * 4)
                    .transform((256, 256), Image.AFFINE, (1, shear / 10, 0, 0, 1, 0), Image.BICUBIC)
            })
#weave.show(ims)

In [ ]:
plot = weave.panels.Plot(ims)
plot.set_x(lambda row: row['rotate'])
plot.set_y(lambda row: row['shear'])
weave.show(plot)

## Facet is fun!

In [ ]:
# TODO: make it work when comparing multiple

facet = weave.panels.Facet(ims)
facet.set_x(lambda row: row['rotate'])
facet.set_y(lambda row: row['shear'])

# TODO: get max count and divide
# TODO: add PanelLayer and PanelColor and color these cells?

# Should PanelNumber be an op, and this be called set Panel?
# No need to pass defaults to PanelNumber, there are none.

#print('CELL INPUT TYPE', facet.var_cell_input.type)
#var_node = facet.var_cell_input.item_var
#grouped_cell_input = facet.var_cell_input.groupby(lambda row: row['y'])
from weave import graph
def make_cell_plot(cell):
    grouped_cell_input = cell.groupby(
        graph.ConstNode(
            weave.types.Function({'row': facet.var_cell_input.type}, weave.types.Int()),
            facet.var_cell_input.var_item['y']))
    p = weave.panels.Plot(grouped_cell_input)
    p.set_x(lambda group: group.count())
    p.set_y(lambda group: group.key())
    p.set_label(lambda group: group.key())
    p.set_mark('bar')
    p.set_no_axes()
    p.set_no_legend()
    return p

facet.set_select(make_cell_plot)

# TODO: make detail / hover work
facet.set_detail(lambda row: row[0]['image'])
weave.show(facet, height=600)

# Next up:
#   - need to be able to group the panel result to Stacey's thing.
#   - figure out how to declare all this can code
#   - can we do the grouping without enabling the PanelPlot grouping?
#   - can we make the plot handle single and multiple nicely?

In [5]:
plot = weave.panels.Plot(ims)
plot.set_x(lambda row: row['rotate'])
plot.groupby_x()
plot.set_y(lambda row: row.count())
weave.show(plot, height=600)

In [13]:
facet = weave.panels.Facet(ims)
facet.set_x(lambda row: row['rotate'])
facet.set_y(lambda row: row['shear'])
p = weave.panels.Plot(facet.var_cell_input)
p.set_y(lambda cell: cell['y'])
p.groupby_y()

# Unfortunate, need to set groupby first here.
# TODO: fix table_state.py code reconfigure types when grouping changes!
p.groupby_label()
p.set_label(lambda cell: cell['y'])
p.set_x(lambda cell: cell.count())
p.set_mark('bar')
p.set_no_axes()
p.set_no_legend()
facet.set_select(lambda cell: p)
weave.show(facet, height=600)


In [ ]:
# TODO: make it work when comparing multiple

facet = weave.panels.Facet(ims)
facet.set_x(lambda row: row['rotate'])
facet.set_y(lambda row: row['shear'])
facet.set_select(lambda row: row)

# TODO: make detail / hover work
facet.set_detail(lambda row: row[0]['image'])
weave.show(facet, height=600)

In [ ]:
# What's the Python / pandas way of doing the above (columnar)
facet = weave.panels.Facet(ims)
# What is this actually doing?
# Its mapping(ims) into the name space of PanelFacet
#   So a ConfigEditor is a component that helps me convert I into InputType of a given panel.
# But what we want is like:
# ims.map(lambda im: {
#   'x': im['rotate'],
#   'y': im['shear'],
#   'im': im['image']
# })
# OR

# This is really nice. You just need to map into the input shape that the panel wants.
#    And we can provide UI mappers that help you figure out what to do.
# grouped = ims.groupby(lambda im: {
#   'x': im['rotate'],
#   'y': im['shear'],
# }).with_col(lambda group: {'count': group.count()})
# input = grouped
#   .with_col(lambda row: {'cell': row['count'] / grouped['count'].sum()})
# PanelGrid(input)
# Hmm
#  columnar form of above would even better!
# input = ims.groupby(im['rotate'], im['shear']).count()
# input['cell'] = grouped['count'] / grouped['count'].sum()
# PanelGrid(input)


# grouped = ims.groupby(lambda im: {
#   'x': im['rotate'],
#   'y': im['shear'],
# }).with_col(lambda group: {'count': group.count()})
# input = grouped
#   .with_col(lambda row: {'cell': row['count'] / grouped['count'].sum()})
# Grid(input)

def PanelConfusion(inp: list[T extends TypedDict], guess_col: keyof T, truth_col: keyof T) -> Panel:
    p = PanelFacet()
    p.set_x(lambda row: ims['guess'])
    p.set_y(lambda row: ims['truth'])
    p.set_select(lambda cell: cell.count(), PanelLayer(PanelColor(), PanelNumber()))
    p.set_post_select(lambda cell: cell / facet.cells.sum())
    p.set_panel(PanelLayer(PanelColor(), PanelNumber()))
    p.set_on_hover()  # Enables hover (defaults)
    # Input is list[image], but we don't force to specify the list handler
    # we just guess a good one. The user can reconfigure in the UI if we're wrong
    p.set_on_hover(PanelImage())
    return p

class PanelConfusionRowType(types.TypedDict):
    guess: types.Union[str, int]
    truth: types.Union[str, int]
    run: types.Optional[types.Union[]]

# TODO: work out how to handle config here.
@op()
def panel_confusion_compare(inp: list[PanelConfusionRowType], defaults: {}) -> Panel:
    f = PanelFacet()
    f.set_x(lambda row: ims['guess'])
    f.set_y(lambda row: ims['truth'])
    f.set_cell_panel(PanelPlot({
        'x': lambda cell: cell.count(),
        # TODO: does this work? How will this f show up in the UI expression? We need to make it a variable...
        # So maybe facet.cells() is a variable?
        'max_x': f.panel.x.max(),
        'y': lambda cell: cell['run'],
        'color': lambda cell: cell['run']
    })
    # TODO: on hover: show images that confused the most runs first?
    f.set_on_hover(PanelImage())
    return f


# OR
facet = weave.panels.Facet(ims, {
    'x': ims['rotate'],
    'y': ims['shear'],
    'select': lambda cell: cell.count(), PanelLayer(PanelColor(), PanelNumber()),
    'on_hover': PanelImage()
})

# But how do we get select color to be normalized?

## Create some ops to make it dynamic and interactive

In [ ]:
class ParamType(typing.TypedDict):
    rotate: int
    shear: int
        
@weave.op(render_info={'type': 'function'})
def make_params(n_rotate: int, n_shear: int) -> list[ParamType]:
    params = []
    for rotate, shear in itertools.product(range(n_rotate), range(n_shear)):
        params.append({
            'rotate': rotate,
            'shear': shear
        })
    return params

@weave.op(render_info={'type': 'function'})
def make_image(rotate: int, shear: int) -> Image.Image:
    return (Image.linear_gradient('L')
        .rotate(rotate * 4)
        .transform((256, 256), Image.AFFINE, (1, shear / 10, 0, 0, 1, 0), Image.BICUBIC))

facet = weave.panels.Facet(make_params(10, 5))
facet.set_x(lambda row: row['rotate'])
facet.set_y(lambda row: row['shear']) 
facet.set_select(lambda row: make_image(row[0]['rotate'], row[0]['shear']))
weave.show(facet, height=600)